# Классификация на эмбеддингах

In [11]:
import torch
import transformers
import numpy as np
import pandas as pd
from tqdm import notebook
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [6]:
# чтение файла
df_tweets = pd.read_csv('/datasets/tweets.csv')

# выдержка из 400 случайных объектов
df_tweets = df_tweets.sample(400).reset_index(drop=True)

In [7]:
# инициализация токенизатора
tokenizer = transformers.BertTokenizer(vocab_file='/datasets/ds_bert/vocab.txt')

# токенизация
tokenized = df_tweets['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# поиск макс. длины токена
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# выравнивание токенов по одной длинне
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

# создание маски
attention_mask = np.where(padded != 0, 1, 0)

In [8]:
# инициализируем конфигурацию BertConfig
config = transformers.BertConfig.from_json_file('/datasets/ds_bert/bert_config.json')

# инициализируем саму модель класса BertModel + файл с предобученной моделью и конфигурацией
model = transformers.BertModel.from_pretrained('/datasets/ds_bert/rubert_model.bin', config=config)

Some weights of the model checkpoint at /datasets/ds_bert/rubert_model.bin were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# кодирование слов в векторы (энбеддинги)
batch_size = 100
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
# выделение признаков
features = np.concatenate(embeddings)
target = df_tweets['positive']

# обучение и протестирование модель
LR_model = LogisticRegression()
LR_model.fit(features, target)
prediction = LR_model.predict(features)
print(accuracy_score(target, prediction))

1.0


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
